# ML Pipeline Preparation

In [101]:
import pandas as pd
import numpy as np
import re

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

import sqlite3
import sqlalchemy as sqldb

import pickle

In [95]:
# loading data from database
connection = sqlite3.connect('../data/02_trusted/worldbank.db')

df = pd.read_sql("SELECT * FROM messages_dataset", con=connection, index_col='index')
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
index,,,,,,,,,,,,,,,,,,,,,
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [96]:
# spliting the data between X and y
X = df['message']
y = df.drop(['id','message','original','genre'], axis=1)
print(y.shape)

def load_data():
    connection = sqlite3.connect('../data/02_trusted/worldbank.db')
    df = pd.read_sql("SELECT * FROM messages_dataset", con=connection, index_col='index')
    X = df['message']
    y = df.drop(['id','message','original','genre'], axis=1)
    return X, y


(26216, 36)


In [97]:
# writing a tokenization function to process your text data
def tokenize(text):
    # cleaning and normalizing
    text = re.sub("\W",' ',text.lower())
    # tokenizing
    words = word_tokenize(text)
    # removing stop words
    words = np.array(words)
    words = words[~np.isin(words,stopwords.words('english'))]
    # lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return words

In [98]:
tokenize(X.iloc[900])

['always',
 'commune',
 'gressier',
 'section',
 '24th',
 'note',
 'incomplete',
 'ask',
 'detail',
 'help']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [99]:
def build_pipeline():
    pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer(smooth_idf=False)),
        ('clf',MultiOutputClassifier(RandomForestClassifier()))
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [100]:
X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline  = build_pipeline()

pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)

KeyboardInterrupt: 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
columns = y_test.columns
for i, col in enumerate(columns):
    print(col)
    print(classification_report(y_test[col], y_pred[:,i]))
    print('___________________________________')


related
              precision    recall  f1-score   support

           0       0.70      0.41      0.52      1511
           1       0.84      0.95      0.89      5000
           2       0.67      0.23      0.34        43

    accuracy                           0.82      6554
   macro avg       0.73      0.53      0.58      6554
weighted avg       0.80      0.82      0.80      6554

___________________________________
request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5415
           1       0.84      0.52      0.64      1139

    accuracy                           0.90      6554
   macro avg       0.87      0.75      0.79      6554
weighted avg       0.90      0.90      0.89      6554

___________________________________
offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6520
           1       0.00      0.00      0.00        34

    accuracy                      

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5708
           1       0.57      0.03      0.06       846

    accuracy                           0.87      6554
   macro avg       0.72      0.51      0.50      6554
weighted avg       0.84      0.87      0.82      6554

___________________________________
infrastructure_related
              precision    recall  f1-score   support

           0       0.93      1.00      0.97      6118
           1       0.00      0.00      0.00       436

    accuracy                           0.93      6554
   macro avg       0.47      0.50      0.48      6554
weighted avg       0.87      0.93      0.90      6554

___________________________________
transport
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6248
           1       0.77      0.09      0.16       306

    accuracy                           0.96      6554
   macro avg       0.86

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001E3FD74D708>)),
  ('tfidf', TfidfTransformer(smooth_idf=False)),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000001E3FD74D708>),
 'tfidf': TfidfTransformer(smooth_idf=False),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': False

In [ ]:
def build_model():
    pipeline = build_pipeline()

    # specify parameters for grid search
    parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]
    }

    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
def display_results(cv,y_test,y_pred):
    columns = y_test.columns
    for i, col in enumerate(columns):
        print(col)
        print(classification_report(y_test[col], y_pred[:,i]))
        print('___________________________________')
        
    print("\nBest Parameters:", cv.best_params_)

    

def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model = build_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    display_results(model, y_test, y_pred)

    return model


model  = main()

KeyboardInterrupt: 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
filename = 'random_forest.sav'
patch = '../data/03_models/'
pickle.dump(model, open(patch+filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.